<a href="https://colab.research.google.com/github/pranalibose/LLM_Workshop/blob/main/LLM_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate rouge_score accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.2

In [ ]:
import transformers

transformers.__version__

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# print a couple of datasets from HF

from huggingface_hub import list_datasets

datasets_list = list_datasets()



In [ ]:
from datasets import load_dataset

cnn_news_summary_ds = load_dataset("cnn_dailymail", "3.0.0")

cnn_news_summary_ds

In [ ]:
# Load a subset of the dataset as we will be working with less data (1%)


In [2]:
# look and shape and features of the dataset which we are going to work with


In [ ]:
cnn_news_summary_ds = cnn_news_summary_ds.train_test_split(test_size = 0.2, seed=42)

cnn_news_summary_ds

In [ ]:
def clean_text(example):
    for txt in ['article', 'highlights']:
        example[txt] = example[txt].replace('\n', '')
        example[txt] = example[txt].replace('--', '')
        example[txt] = example[txt].replace('\\', '')
        example[txt] = example[txt].replace('/', '')
        example[txt] = example[txt].replace('"', '')
        example[txt] = example[txt].replace('``', '')
        return example

In [ ]:
# call the clean_text function on each and every record of our dataset


In [ ]:
TEXT_INDEX = 33

example_text = cleaned_cnn_news_summary_ds['test']['article'][TEXT_INDEX]

example_text

# Using Pre-trained model to extract summaries

In [ ]:
from transformers import pipeline

# set the model you want to use
MODEL_NAME = ''

prefix = 'summarize: '

# add the correct parameters to the transformers pipeline
summarizer = pipeline(task='', model='')

summary_text = summarizer(prefix + example_text)

summary_text

In [ ]:
ref_text = cleaned_cnn_news_summary_ds['test']['highlights'][TEXT_INDEX]

ref_text

# Evaluation of summarization using ROUGE

In [ ]:
import evaluate

rouge = evaluate.load('rouge')

rouge

In [ ]:
# add the predictions and references to compute the rouge scores

rouge_score = rouge.compute(predictions=[], references=[], use_stemmer=True)

rouge_score

# Generate summaries for multiple articles in the dataset

In [ ]:
article_texts = cleaned_cnn_news_summary_ds['test']['article']

article_summaries = cleaned_cnn_news_summary_ds['test']['highlights']


In [ ]:
# Execution time ~ 1 sec

from tqdm import tqdm

candidate_summaries = []

prefix = 'summarize: '

for i, text in enumerate(tqdm(article_texts[:10])):
    candidate = summarizer(prefix + text)
    candidate_summaries.append(candidate[0]['summary_text'])

In [ ]:
result_agg = rouge.compute(predictions=candidate_summaries, references=article_summaries[:10], use_stemmer=True)

result_agg

In [ ]:
result_unagg = rouge.compute(predictions=candidate_summaries, references=article_summaries[:10], use_stemmer=True, use_aggregator=False)

result_unagg

In [ ]:
import numpy as np

# test the best and worst scores of rouge 2
result_unagg_rsum = np.array(result_unagg['rouge2'])


In [ ]:
import pandas as pd

act_vs_pred_summaries_df = pd.DataFrame(list(zip(candidate_summaries, article_summaries[:10])),
                                        columns = ['Predicted_summaries', 'Reference_summaries'])

act_vs_pred_summaries_df.head()

In [ ]:
# compare the best scored summaries

print('Actual Summary')
print(act_vs_pred_summaries_df._get_value(3, 'Predicted_summaries'))
print()
print('Predicted Summary')
print(act_vs_pred_summaries_df._get_value(3, 'Reference_summaries'))

In [ ]:
# compare the worst scored summaries


# Fine-Tuning T5 Model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = MODEL_NAME)

tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
prefix = 'summarize: '

def preprocess_function(examples):

    inputs = [prefix + doc for doc in examples['article']]
    model_inputs = tokenizer(inputs, max_length = 1024, truncation = True)

    labels = tokenizer(text_target = examples['highlights'], max_length = 128, truncation = True)

    model_inputs['labels'] = labels['input_ids']

    return model_inputs

In [ ]:
# call the preprocess function on all of the records of our cleaned_cnn_news_summary_ds, use batched = True


In [ ]:
# function to compute the rouge scores and evaluate the model's performance

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens = True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens = True)

    result = rouge.compute(predictions = decoded_preds, references = decoded_labels, use_stemmer = True)

    prediction_length = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result['generated_length'] = np.mean(prediction_length)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# fill in the missing parameters: output_dir, num_train_epochs, push_to_hub

# training_args = Seq2SeqTrainingArguments(
#     output_dir = '',
#     eval_strategy = 'epoch',
#     learning_rate = 2e-5,
#     per_device_train_batch_size = 16,
#     per_device_eval_batch_size = 16,
#     weight_decay = 0.01,
#     save_total_limit = 3,
#     num_train_epochs = ,
#     predict_with_generate = True,
#     fp16 = True,
#     push_to_hub =
# )

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_cnn_news_summary_ds['train'],
    eval_dataset = tokenized_cnn_news_summary_ds['test'],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

# train the trainer

In [ ]:
# push the configurations of our fine tuned model to HF hub
